In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import os

C:\Users\amartinezgil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Definir la ruta al directorio donde se extrajeron los archivos
docs_folder = 'docs'

# Función para listar y leer archivos Markdown en español
def cargar_y_combinar_markdown_en_espanol(directorio):
    contenido = []
    for root, _, files in os.walk(directorio):
        for file in files:
            if file.endswith(".es.md"):  # Seleccionar solo archivos en español
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    contenido.append(f.read())
    return "\n\n".join(contenido)

# Cargar y combinar contenido
contenido_documentacion = cargar_y_combinar_markdown_en_espanol(docs_folder)

texts = contenido_documentacion.split("\n\n")  # Dividir por documentos o secciones, si es aplicable

In [3]:
# Crear un objeto Dataset con el contenido dividido
dataset = Dataset.from_dict({"text": texts})

# Dividir el dataset en entrenamiento y test (o validación)
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [4]:
# Tokenizar los datos
model_checkpoint = "gpt2"  # Modelo base
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Establecer el token de EOS como token de padding
tokenizer.pad_token = tokenizer.eos_token

def tokenize_and_prepare_labels(examples):
    # Tokenizar el texto
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    # Asegurarse de que las etiquetas sean los input_ids desplazados
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    
    # Aquí asumimos que eos_token_id es el mismo que pad_token_id, ajusta según sea necesario
    pad_token_id = tokenizer.pad_token_id

    # Desplazar las etiquetas una posición a la derecha y añadir el pad token al inicio
    tokenized_inputs["labels"] = [[pad_token_id] + label[:-1] for label in tokenized_inputs["labels"]]
    
    return tokenized_inputs

# Aplicar esta función al dataset
tokenized_train = train_dataset.map(tokenize_and_prepare_labels, batched=True)
tokenized_test = test_dataset.map(tokenize_and_prepare_labels, batched=True)

Map: 100%|██████████| 121/121 [00:00<00:00, 3558.56 examples/s]


In [5]:
# Cargar el modelo preentrenado
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    per_device_train_batch_size=2,  # Ajuste según la capacidad de tu hardware
    per_device_eval_batch_size=2,
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# Entrenar el modelo
trainer.train()

 92%|█████████▏| 500/545 [1:33:04<09:12, 12.28s/it]  

{'loss': 0.1564, 'learning_rate': 1.6513761467889911e-06, 'epoch': 0.92}


                                                   
 92%|█████████▏| 500/545 [1:37:15<09:12, 12.28s/it]

{'eval_loss': 0.006177566479891539, 'eval_runtime': 250.6912, 'eval_samples_per_second': 0.483, 'eval_steps_per_second': 0.243, 'epoch': 0.92}


100%|██████████| 545/545 [1:44:32<00:00, 11.51s/it]  

{'train_runtime': 6272.7509, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.087, 'train_loss': 0.14406084303462177, 'epoch': 1.0}


TrainOutput(global_step=545, training_loss=0.14406084303462177, metrics={'train_runtime': 6272.7509, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.087, 'train_loss': 0.14406084303462177, 'epoch': 1.0})

In [6]:
# Guardar el modelo ajustado
trainer.save_model("/mnt/data/wakamiti_finetuned_model")